In [1]:
import pandas as pd

# create dummies for Gender and countries 

In [17]:
df15 = pd.read_csv('data/marathon_results_2015.csv')
df16 = pd.read_csv('data/marathon_results_2016.csv')
df17 = pd.read_csv('data/marathon_results_2017.csv')
df18 = pd.read_csv('data/marathon_results_2018.csv')
df18.rename(columns={'Blank': 'Unnamed'}, inplace = True)
print(df18.shape)
df18.head()

(24087, 26)


,Unnamed: 0,Unnamed: 0.1,Bib,Name,Age,M/F,City,State,Country,Citizen,...,25K,30K,35K,40K,Pace,Proj. Time,Official Time,Overall,Gender,Division
0,0,0,189,"Ash, Alan",26,M,Atmore,AL,USA,NaN,...,1:31:16,1:50:19,2:09:22,2:28:08,0:05:58,NaN,2:36:04,63,63,59
1,1,1,855,"Goodman, John Mark",38,M,Birmingham,AL,USA,NaN,...,1:34:33,1:53:49,2:13:27,2:33:06,0:06:11,NaN,2:41:52,187,186,173
2,2,2,1049,"Mcclung, Joshua",34,M,Birmingham,AL,USA,NaN,...,1:35:31,1:55:14,2:15:14,2:34:45,0:06:15,NaN,2:43:42,247,246,226
3,3,3,303,"Graham, James",26,M,Birmingham,AL,USA,NaN,...,1:33:09,1:52:29,2:13:49,2:37:17,0:06:23,NaN,2:47:07,385,377,346
4,4,4,5267,"Pierce, Nathaniel",31,M,Huntsville,AL,USA,NaN,...,1:40:23,2:00:42,2:21:22,2:41:26,0:06:30,NaN,2:50:19,608,593,536


In [18]:
import numpy as np

def seconds_converter(time):
    '''Takes a time in the string form '00:00:00' and converts it into integer 
    of seconds.  Handles special '-' cases and converts them to NaNs '''
    
    if time == '-':
        return np.nan
    
    time_split = [int(x) for x in(time.split(':'))]
    return time_split[0]*(60**2) + time_split[1]*60 + time_split[2] 

def best_fit_slope(x,y):
    '''Calculate the best fit slope for all 5K splits.'''
    m = (((x.mean()*y.mean()) - (x*y).mean()) / 
         (x.mean()**2 - (x**2).mean()))
    return round(m, 2)


def race_slope(row):
    '''Create a new column of the race progression slope.'''
    nK_splits = ['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K']
    x = np.array([5, 10, 15, 20, 25, 30, 35, 40])
    y = np.array([row['5K']] + list(np.diff(row[nK_splits])))
    
    return best_fit_slope(x,y)

def add_weather_data(df, year):
    '''Add the weather data for the corresponding year.'''
    
    year = int(year)
    median = df['Official Time'].median()/3600

    pre_median_2015 = {'temp': 48, 'dew_point': 39, 'humidity': 0.71, 'wind': 17, 'gusts': 26}
    post_median_2015 = {'temp': 44, 'dew_point': 41, 'humidity': 0.89,'wind': 16, 'gusts': 0}
    pre_median_2016 = {'temp': 64, 'dew_point': 31, 'humidity': 0.29, 'wind': 14, 'gusts': 22}
    post_median_2016 = {'temp': 53, 'dew_point': 37, 'humidity': 0.55, 'wind': 14, 'gusts': 0}
    pre_median_2017 = {'temp': 74, 'dew_point': 40, 'humidity': 0.29, 'wind': 21, 'gusts': 29}
    post_median_2017 = {'temp': 73, 'dew_point': 34, 'humidity': 0.24, 'wind': 22, 'gusts': 28}
    pre_median_2018 = {'temp': 45, 'dew_point': 43, 'humidity': 0.93, 'wind': 14, 'gusts': 23}
    post_median_2018 = {'temp': 44, 'dew_point': 44, 'humidity': 1.00, 'wind': 17, 'gusts': 0}
    
    new_cols = ['temp', 'dew_point', 'humidity', 'wind', 'gusts']
    
    for col in new_cols:
        if year == 2015:
            df[col] = [pre_median_2015[col] if x < median else post_median_2015[col] for x in df['Official Time']]
        elif year == 2016:
            df[col] = [pre_median_2016[col] if x < median else post_median_2016[col] for x in df['Official Time']]
        elif year == 2017:
            df[col] = [pre_median_2017[col] if x < median else post_median_2017[col] for x in df['Official Time']]
        else:
            df[col] = [pre_median_2018[col] if x < median else post_median_2018[col] for x in df['Official Time']]

    return df


def clean_data(df, year):
    '''Takes in a DataFrame, drops unnecessary columns, and applies seconds_converter. '''
    
    # drop unnecessary columns
    df.drop(list(df.filter(regex='Unnamed')), axis=1, inplace=True)
    df.drop(['Citizen', 'Proj. Time', 'City'], axis=1, inplace=True)
    
    # rename columns for clarity
    df.rename(columns={'Overall': 'overall_rank', 'Gender': 'gender_rank', 
                         'Division': 'division_rank', 'M/F': 'Gender'}, inplace = True)
    
    # convert the time to seconds on these columns
    time_cols = ['5K', '10K', '15K', '20K', 'Half', '25K', 
             '30K', '35K', '40K', 'Pace', 'Official Time']
    for col in time_cols:
        df[col] = df[col].apply(seconds_converter)
        
    # fill the State column with NaNs to 'none' instead so it can be 
    # concatenated with the country column
    df['State'].fillna('', inplace=True)
    df['state_country'] = df['State'] + '_' + df['Country']
    
    # create a new column progression_slope
    df['progression_slope'] = df.apply(race_slope, axis=1) 
    
    df = add_weather_data(df, year)
    
    df.drop(['State', 'Country', '5K', '10K', '15K', '20K', '25K', 
             '30K', '35K', '40K'], axis=1, inplace=True)
 
    return df


In [20]:
df18 = clean_data(df18, 2018)
df18.head()

,Bib,Name,Age,Gender,Half,Pace,Official Time,overall_rank,gender_rank,division_rank,state_country,progression_slope,temp,dew_point,humidity,wind,gusts
0,189,"Ash, Alan",26,M,4606.0,358,9364,63,63,59,AL_USA,1.84,44,44,1.0,17,0
1,855,"Goodman, John Mark",38,M,4788.0,371,9712,187,186,173,AL_USA,1.51,44,44,1.0,17,0
2,1049,"Mcclung, Joshua",34,M,4805.0,375,9822,247,246,226,AL_USA,1.98,44,44,1.0,17,0
3,303,"Graham, James",26,M,4713.0,383,10027,385,377,346,AL_USA,6.95,44,44,1.0,17,0
4,5267,"Pierce, Nathaniel",31,M,5079.0,390,10219,608,593,536,AL_USA,0.48,44,44,1.0,17,0


In [ ]:
median = df18['Official Time'].median()/3600




pre_median_2018 = {'temp': 45, 'dew point': 43, 'humidity': 0.93, 'wind': 14, 'gusts': 23}
post_median_2018 = {'temp': 44, 'dew point': 44, 'humidity': 1.00, 'wind': 17, 'gusts': 0}

df18['temp'] = [pre_median_2018['temp'] if x < median else post_median_2018['temp'] for x in df18['Official Time']]
df18['dew_point'] = [pre_median_2018['dew point'] if x < median else post_median_2018['dew point'] for x in df18['Official Time']]
df18['humidity'] = [pre_median_2018['humidity'] if x < median else post_median_2018['humidity'] for x in df18['Official Time']]
df18['wind'] = [pre_median_2018['wind'] if x < median else post_median_2018['wind'] for x in df18['Official Time']]
df18['gusts'] = [pre_median_2018['gusts'] if x < median else post_median_2018['gusts'] for x in df18['Official Time']]

df18.head()

In [14]:
def add_weather_data(df, year):
    '''Add the weather data for the corresponding year.'''

    median = df['Official Time'].median()/3600

    pre_median_2015 = {'temp': 48, 'dew_point': 39, 'humidity': 0.71, 'wind': 17, 'gusts': 26}
    post_median_2015 = {'temp': 44, 'dew_point': 41, 'humidity': 0.89,'wind': 16, 'gusts': 0}
    pre_median_2016 = {'temp': 64, 'dew_point': 31, 'humidity': 0.29, 'wind': 14, 'gusts': 22}
    post_median_2016 = {'temp': 53, 'dew_point': 37, 'humidity': 0.55, 'wind': 14, 'gusts': 0}
    pre_median_2017 = {'temp': 74, 'dew_point': 40, 'humidity': 0.29, 'wind': 21, 'gusts': 29}
    post_median_2017 = {'temp': 73, 'dew_point': 34, 'humidity': 0.24, 'wind': 22, 'gusts': 28}
    pre_median_2018 = {'temp': 45, 'dew_point': 43, 'humidity': 0.93, 'wind': 14, 'gusts': 23}
    post_median_2018 = {'temp': 44, 'dew_point': 44, 'humidity': 1.00, 'wind': 17, 'gusts': 0}
    
    new_cols = ['temp', 'dew_point', 'humidity', 'wind', 'gusts']
    
    for col in new_cols:
        if year == 2015:
            df[col] = [pre_median_2015[col] if x < median else post_median_2015[col] for x in df['Official Time']]
        elif year == 2016:
            df[col] = [pre_median_2016[col] if x < median else post_median_2016[col] for x in df['Official Time']]
        elif year == 2017:
            df[col] = [pre_median_2017[col] if x < median else post_median_2017[col] for x in df['Official Time']]
        else:
            df[col] = [pre_median_2018[col] if x < median else post_median_2018[col] for x in df['Official Time']]

    return df

In [15]:
add_weather_data(df18, 2018)

,Bib,Name,Age,Gender,Half,Pace,Official Time,overall_rank,gender_rank,division_rank,state_country,progression_slope,temp,dew_point,humidity,wind,gusts
0,189,"Ash, Alan",26,M,4606.0,358,9364,63,63,59,AL_USA,1.84,44,44,1.0,17,0
1,855,"Goodman, John Mark",38,M,4788.0,371,9712,187,186,173,AL_USA,1.51,44,44,1.0,17,0
2,1049,"Mcclung, Joshua",34,M,4805.0,375,9822,247,246,226,AL_USA,1.98,44,44,1.0,17,0
3,303,"Graham, James",26,M,4713.0,383,10027,385,377,346,AL_USA,6.95,44,44,1.0,17,0
4,5267,"Pierce, Nathaniel",31,M,5079.0,390,10219,608,593,536,AL_USA,0.48,44,44,1.0,17,0
5,853,"Sillivant, Daniel",31,M,4741.0,399,10458,953,926,808,AL_USA,8.74,44,44,1.0,17,0
6,4962,"Malloy, Timothy",28,M,5227.0,400,10469,972,944,825,AL_USA,-0.34,44,44,1.0,17,0
7,1317,"Bradley, Owen",38,M,4990.0,403,10547,1137,1102,951,AL_USA,NaN,44,44,1.0,17,0
8,1271,"Neiman, John",43,M,4973.0,404,10574,1199,1162,113,AL_USA,NaN,44,44,1.0,17,0
9,2067,"Kidd, Elena",32,F,5263.0,404,10588,1227,42,39,AL_USA,0.01,44,44,1.0,17,0
